In [1]:
import (
	"fmt"
	"log"
	"os"
	"time"

	"github.com/evertoncolling/poc-requests-go/pkg/api"
	"github.com/evertoncolling/poc-requests-go/pkg/dto"

	grob "github.com/MetalBlueberry/go-plotly/graph_objects"
	"github.com/janpfeifer/gonb/gonbui/plotly"
	"github.com/joho/godotenv"
)

// Find a unit by external ID
func findUnitByExternalId(unitList *dto.UnitList, externalId string) (*dto.Unit, error) {
	for _, unit := range unitList.Items {
		if unit.ExternalId == externalId {
			return &unit, nil
		}
	}
	return nil, fmt.Errorf("unit not found with ExternalId: %s", externalId)
}

func getClient() api.CogniteClient {
    // Load environment variables
    err := godotenv.Load(".env")
    if err != nil {
        log.Fatalf("Error loading .env file")
    }

    // Read client credentials from environment variables
    clientID := os.Getenv("CLIENT_ID")
    clientSecret := os.Getenv("CLIENT_SECRET")
    tenantID := os.Getenv("TENANT_ID")
    cluster := os.Getenv("CDF_CLUSTER")
    project := os.Getenv("CDF_PROJECT")

    credentials := api.AzureADClientCredentials(
        clientID,
        clientSecret,
        tenantID,
        cluster,
    )
    clientConfig := api.ClientConfig{
        ClientName:  "poc-requests-go",
        Cluster:     cluster,
        Project:     project,
        Credentials: credentials,
    }
    client := api.NewCogniteClient(clientConfig)
    return client
}

// initialize client
var client = getClient()

%%
fmt.Println("### Testing fetching some time series")

// List time series
tsList, err := client.TimeSeries.List(100, false, "", "", nil, nil, "")
if err != nil {
	fmt.Println("Error:", err)
	return
}
fmt.Println("Time Series Count:", len(tsList.Items))

### Testing fetching some time series
Time Series Count: 100


In [2]:
%%
// Filter time series
filter := &dto.TimeSeriesFilter{
	UnitQuantity: "Pressure",
}
filteredTsList, err := client.TimeSeries.Filter(filter, nil, 100, "", "", nil)
if err != nil {
	fmt.Println("Error:", err)
	return
}

fmt.Println("Filtered Time Series Count:", len(filteredTsList.Items))

Filtered Time Series Count: 100


In [3]:
func getUnitList() dto.UnitList {
	unitList, err := client.Units.List()
	if err != nil {
		fmt.Println("Error:", err)
		return dto.UnitList{}
	}
	return unitList
}
var unitList = getUnitList()

%%
fmt.Println("### Testing fetching the Unit catalog")
fmt.Println("Unit Count:", len(unitList.Items))

### Testing fetching the Unit catalog
Unit Count: 318


In [4]:
%%
// Fetch the latest data points
fmt.Println("### Latest Data Points")
externalIds := []string{"EVE-TI-FORNEBU-01-2", "EVE-TI-FORNEBU-01-3"}
var latestDataPointsQueryItems []dto.LatestDataPointsQueryItem
for _, externalId := range externalIds {
	latestDataPointsQueryItems = append(latestDataPointsQueryItems, dto.LatestDataPointsQueryItem{
		ExternalId: externalId,
	})
}
latestDataPoints, err := client.TimeSeries.RetrieveLatest(
	&latestDataPointsQueryItems,
	nil,
)
if err != nil {
	fmt.Println("Error:", err)
	return
}
for _, latestDataPoint := range latestDataPoints.Items {
	unit, err := findUnitByExternalId(&unitList, latestDataPoint.UnitExternalId)
	if err != nil {
		fmt.Println("Error:", err)
		return
	}
	fmt.Println(latestDataPoint.ExternalId+" ["+unit.Symbol+"]"+":", latestDataPoint.DatapointType)
}

### Latest Data Points
EVE-TI-FORNEBU-01-2 [°C]: &{datapoints:{timestamp:1738092000000  value:3.1}}
EVE-TI-FORNEBU-01-3 [°C]: &{datapoints:{timestamp:1738092003000  value:3.23}}


In [5]:
%%
// Get many data points (performance test)
fmt.Println("### Data Points (performance test)")
items := []dto.DataPointsQueryItem{
	{
		ExternalId: "EVE-TI-FORNEBU-01-2",
		Start:      "300d-ago",
		End:        "now",
		Limit:      100000,
	},
}
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}
dps := dataPoints.Items[0]
fmt.Println("Data Points External ID:", dps.ExternalId)
unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}
fmt.Println("Data Points Unit:", dps.UnitExternalId, " -> ["+unit.Symbol+"]")
// not sure if there is a simpler way to get to the data points :)
switch v := (*dps).DatapointType.(type) {
case *dto.DataPointListItem_NumericDatapoints:
	fmt.Println("Data Points Count:", len(v.NumericDatapoints.Datapoints))
case *dto.DataPointListItem_StringDatapoints:
	fmt.Println("Data Points Count:", len(v.StringDatapoints.Datapoints))
case *dto.DataPointListItem_AggregateDatapoints:
	fmt.Println("Data Points Count:", len(v.AggregateDatapoints.Datapoints))
default:
	fmt.Println("Unknown data point type:", v)
}
fmt.Printf("Time taken: %s\n", elapsed)

### Data Points (performance test)
Data Points External ID: EVE-TI-FORNEBU-01-2
Data Points Unit: temperature:deg_c  -> [°C]
Data Points Count: 82493
Time taken: 181.524792ms


In [49]:
%%
// let's try a more reasonable number of data points
fmt.Println("### Data Points (for plotting)")
items := []dto.DataPointsQueryItem{
	{
		ExternalId:  "EVE-TI-FORNEBU-01-2",
		Start:       "300d-ago",
		End:         "now",
		Aggregates:  []string{"average"},
		Granularity: "1h",
		Limit:       10000,
	},
}
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Prepare the data for plotting
dps := dataPoints.Items[0]
aggregateDatapointsType := dps.DatapointType.(*dto.DataPointListItem_AggregateDatapoints)
aggregateDatapoints := aggregateDatapointsType.AggregateDatapoints.Datapoints
values := make([]float64, 0, len(aggregateDatapoints))
timestamps := make([]time.Time, 0, len(aggregateDatapoints))
for _, datapoint := range aggregateDatapoints {
	if datapoint != nil {
		values = append(values, datapoint.Average)
		timestamp := time.Unix(0, datapoint.Timestamp*int64(time.Millisecond))
		timestamps = append(timestamps, timestamp)
	}
}
fmt.Printf("Time taken: %s\n", elapsed)
unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Plot the data
// Plot the data
fig := &grob.Fig{
    Data: grob.Traces{
        &grob.Scatter{
            X:    timestamps,
            Y:    values,
            Name: dps.ExternalId,
            Line: &grob.ScatterLine{
                Color: "#6b9eff",  // Light blue color for the line
                Width: 1,
            },
        },
    },
    Layout: &grob.Layout{
        Title: &grob.LayoutTitle{
            Text: fmt.Sprintf("Fetched %d data points - hourly aggregates", len(values)),
            Font: &grob.LayoutTitleFont{
                Size:  24.0,
                Color: "#ffffff",  // White text
            },
        },
        PaperBgcolor: "rgba(0,0,0,0)",  // Transparent background
        PlotBgcolor:  "rgba(0,0,0,0)",  // Transparent plot area
        Font: &grob.LayoutFont{
            Family: "Helvetica",
            Size:   12,
            Color:  "#ffffff",  // White text
        },
        Xaxis: &grob.LayoutXaxis{
            Showspikes:     grob.True,
            Spikemode:      grob.LayoutXaxisSpikemode("across"),
            Spikethickness: 1.0,
            Spikedash:      "solid",
            Gridcolor:      "rgba(255,255,255,0.1)",  // Subtle grid lines
            Linecolor:      "#ffffff",                // White axis line
            Zerolinecolor:  "#ffffff",                // White zero line
            Tickfont: &grob.LayoutXaxisTickfont{
                Color: "#ffffff",  // White tick labels
            },
        },
        Yaxis: &grob.LayoutYaxis{
            Title: &grob.LayoutYaxisTitle{
                Text: fmt.Sprintf("%s [%s]", unit.Quantity, unit.Symbol),
                Font: &grob.LayoutYaxisTitleFont{
                    Color: "#ffffff",  // White axis title
                },
            },
            Gridcolor:     "rgba(255,255,255,0.1)",  // Subtle grid lines
            Linecolor:     "#ffffff",                // White axis line
            Zerolinecolor: "#ffffff",                // White zero line
            Tickfont: &grob.LayoutYaxisTickfont{
                Color: "#ffffff",  // White tick labels
            },
        },
        Spikedistance: -1,
        Legend: &grob.LayoutLegend{
            Orientation: grob.LayoutLegendOrientation("h"),
            Yanchor:     grob.LayoutLegendYanchor("bottom"),
            Y:           1.02,
            Xanchor:     grob.LayoutLegendXanchor("right"),
            X:           1.0,
            Font: &grob.LayoutLegendFont{
                Color: "#ffffff",  // White legend text
            },
            Bgcolor: "rgba(0,0,0,0)",  // Transparent legend background
        },
        Margin: &grob.LayoutMargin{
            L: 50,  // Left margin
            R: 30,  // Right margin
            T: 50,  // Top margin
            B: 50,  // Bottom margin
        },
        Hovermode:  "x",
        Showlegend: grob.True,
        Height:     600,
        Hoverlabel: &grob.LayoutHoverlabel{
            Bgcolor: "rgba(0,0,0,0.8)",  // Semi-transparent black tooltip background
            Font: &grob.LayoutHoverlabelFont{
                Color: "#ffffff",  // White tooltip text
            },
        },
    },
}

// Optional: Add hover template for better tooltip formatting
fig.Data[0].(*grob.Scatter).Hovertemplate = "%{x}<br>%{y:.2f} °C<extra></extra>"

// Display the plot
plotly.DisplayFig(fig)

### Data Points (for plotting)
Time taken: 158.607125ms


In [52]:
// generateLineItems converts the values to ECharts line data points
func generateLineItems(values []float64) []opts.LineData {
	items := make([]opts.LineData, 0, len(values))
	for _, v := range values {
		items = append(items, opts.LineData{Value: v})
	}
	return items
}

%%
// Define the data points query
items := []dto.DataPointsQueryItem{
	{
		ExternalId:  "EVE-TI-FORNEBU-01-2",
		Start:       "300d-ago",
		End:         "now",
		Aggregates:  []string{"average"},
		Granularity: "1h",
		Limit:       10000,
	},
}

// Retrieve data points
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Prepare the data for plotting
dps := dataPoints.Items[0]
aggregateDatapointsType := dps.DatapointType.(*dto.DataPointListItem_AggregateDatapoints)
aggregateDatapoints := aggregateDatapointsType.AggregateDatapoints.Datapoints
values := make([]float64, 0, len(aggregateDatapoints))
timestamps := make([]time.Time, 0, len(aggregateDatapoints))
for _, datapoint := range aggregateDatapoints {
	if datapoint != nil {
		values = append(values, datapoint.Average)
		timestamp := time.Unix(0, datapoint.Timestamp*int64(time.Millisecond))
		timestamps = append(timestamps, timestamp)
	}
}
fmt.Printf("Time taken: %s\n", elapsed)

unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Convert timestamps to string format for the X-axis
xAxisData := make([]string, 0, len(timestamps))
for _, t := range timestamps {
	xAxisData = append(xAxisData, t.Format("2006-01-02 15:04"))
}

// Create a new line chart
line := charts.NewLine()

// Convert timestamps to formatted strings for x-axis
timeLabels := make([]string, len(timestamps))
for i, ts := range timestamps {
    timeLabels[i] = ts.Format("2006-01-02 15:04")
}

// Convert values to opts.LineData
lineData := make([]opts.LineData, len(values))
for i, v := range values {
    lineData[i] = opts.LineData{Value: float64(int(v*100))/100}  // Round to 2 decimals
}

// Configure the chart
line.SetGlobalOptions(
    charts.WithTitleOpts(opts.Title{
        Title: fmt.Sprintf("Fetched %d data points - hourly aggregates", len(values)),
        Subtitle: dps.ExternalId,
        TitleStyle: &opts.TextStyle{
            Color: "#ffffff",
        },
    }),
    charts.WithTooltipOpts(opts.Tooltip{
        Show: opts.Bool(true),
        Trigger: "axis",
        Formatter: "{b0} <br/> {a0}: {c0}°C",
    }),
    charts.WithGridOpts(opts.Grid{
        Left: "3%",
        Right: "5%",
        Top: "10%",
        Bottom: "10%",
        ContainLabel: opts.Bool(true),
    }),
    charts.WithLegendOpts(opts.Legend{
        Show: opts.Bool(true),
        Top: "3%",
        Right: "5%",
        Orient: "horizontal",
        TextStyle: &opts.TextStyle{
            Color: "#ffffff",
        },
    }),
    charts.WithYAxisOpts(opts.YAxis{
        Name: fmt.Sprintf("%s [%s]", unit.Quantity, unit.Symbol),
        NameLocation: "middle",
        NameGap: 50,
        Type: "value",
        AxisLabel: &opts.AxisLabel{
            Color: "#ffffff",
            Formatter: "{value}", // Corrected axis label format
        },
        AxisLine: &opts.AxisLine{
            LineStyle: &opts.LineStyle{
                Color: "#ffffff",
            },
        },
    }),
    charts.WithXAxisOpts(opts.XAxis{
        Name: "Time",
        AxisLabel: &opts.AxisLabel{
            Color: "#ffffff",
        },
        AxisLine: &opts.AxisLine{
            LineStyle: &opts.LineStyle{
                Color: "#ffffff",
            },
        },
    }),
    charts.WithInitializationOpts(opts.Initialization{
        Width: "1200px",
        Height: "600px",
        BackgroundColor: "transparent",
        Theme: "dark",
    }),
)

// Modify the series to use white color for the line
line.SetXAxis(timeLabels).
    AddSeries(dps.ExternalId,
        lineData,
        charts.WithLineChartOpts(opts.LineChart{
            Smooth: opts.Bool(true),
        }),
        charts.WithLineStyleOpts(opts.LineStyle{
            Color: "#6b9eff",  // Light blue line color
        }),
    )

// If you're using gonb_echarts:
err = gonb_echarts.Display(line, "width: 1200px; height: 800px; background: transparent;")
if err != nil {
    fmt.Printf("Error displaying chart: %v\n", err)
}

Time taken: 175.390834ms
